# Extract sources from catalogs  in  OGA

 work with Weakly_2024_07
- use jupyter kernel LSST
- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2024/03/01
- update : 2024/03/01

- ``Quantum graphs`` : https://tigress-web.princeton.edu/~lkelvin/pipelines/
- ``Quantum graphs FGCM`` : https://tigress-web.princeton.edu/~lkelvin/pipelines/w_2024_07/drp_pipe/LATISS/DRP/pipeline_drp_pipe_LATISS_DRP_step2bcde.pdf

      DatasetType('fgcmFitParameters0', {instrument}, Catalog),
      DatasetType('fgcmMakeLut_config', {}, Config),
      DatasetType('fgcm_standard_passband', {band, instrument, physical_filter}, TransmissionCurve),
      DatasetType('fgcm_standard_atmosphere', {instrument}, TransmissionCurve),
      DatasetType('fgcmLookUpTable', {instrument}, Catalog),
      DatasetType('fgcmMakeLut_metadata', {instrument}, TaskMetadata),
      DatasetType('fgcmMakeLut_log', {instrument}, ButlerLogRecords),
      DatasetType('fgcmBuildFromIsolatedStars_config', {}, Config),
      DatasetType('fgcm_star_ids', {instrument}, ArrowAstropy),
      DatasetType('fgcm_reference_stars', {instrument}, ArrowAstropy),
      DatasetType('fgcmVisitCatalog', {instrument}, Catalog),
      DatasetType('fgcm_star_observations', {instrument}, ArrowAstropy),
      DatasetType('fgcmBuildFromIsolatedStars_metadata', {instrument}, TaskMetadata),
      DatasetType('fgcmBuildFromIsolatedStars_log', {instrument}, ButlerLogRecords),
      DatasetType('fgcmFitCycle_config', {}, Config),
      DatasetType('fgcmFlaggedStars0', {instrument}, Catalog),
      DatasetType('fgcmAtmosphereParameters4', {instrument}, Catalog),
      DatasetType('fgcmFitParameters4', {instrument}, Catalog),
      DatasetType('fgcmZeropoints4', {instrument}, Catalog),
      DatasetType('fgcmFlaggedStars1', {instrument}, Catalog),
      DatasetType('fgcmFlaggedStars2', {instrument}, Catalog),
      DatasetType('fgcmStandardStars4', {instrument}, SimpleCatalog),
      DatasetType('fgcmFitParameters3', {instrument}, Catalog),
      DatasetType('fgcmFitParameters1', {instrument}, Catalog),
      DatasetType('fgcmFlaggedStars4', {instrument}, Catalog),
      DatasetType('fgcmFitParameters2', {instrument}, Catalog),
      DatasetType('fgcmFlaggedStars3', {instrument}, Catalog),
      DatasetType('fgcmFitCycle_metadata', {instrument}, TaskMetadata),
      DatasetType('fgcmFitCycle_log', {instrument}, ButlerLogRecords),
      DatasetType('fgcmStandardStars6', {instrument}, SimpleCatalog),
      DatasetType('fgcmZeropoints6', {instrument}, Catalog),
      DatasetType('fgcmFitParameters5', {instrument}, Catalog),
      DatasetType('fgcmAtmosphereParameters6', {instrument}, Catalog),
      DatasetType('fgcmFlaggedStars6', {instrument}, Catalog),
      DatasetType('fgcmFitParameters6', {instrument}, Catalog),
      DatasetType('fgcmFlaggedStars5', {instrument}, Catalog),
      DatasetType('fgcmFlaggedStars8', {instrument}, Catalog),
      DatasetType('fgcmFlaggedStars7', {instrument}, Catalog),
      DatasetType('fgcmFitParameters8', {instrument}, Catalog),
      DatasetType('fgcmFitParameters9', {instrument}, Catalog),
      DatasetType('fgcmFitParameters7', {instrument}, Catalog),
      DatasetType('fgcmFlaggedStars9', {instrument}, Catalog),
      DatasetType('fgcmOutputProducts_metadata', {instrument}, TaskMetadata),
      DatasetType('transmission_atmosphere_fgcm', {band, instrument, physical_filter, visit}, TransmissionCurve),
      DatasetType('fgcmPhotoCalibCatalog', {band, instrument, physical_filter, visit}, ExposureCatalog),
      DatasetType('fgcmOutputProducts_log', {instrument}, ButlerLogRecords),
      DatasetType('fgcmOutputProducts_config', {}, Config)]

In [1]:
! eups list -s | grep LOCAL

analysis_tools        LOCAL:/home/d/dagoret/repos/repos_w_2024_07/analysis_tools 	setup
atmospec              LOCAL:/home/d/dagoret/repos/repos_w_2024_07/atmospec 	setup
eups                  LOCAL:/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-8.0.0/eups 	setup
spectractor           LOCAL:/home/d/dagoret/repos/repos_w_2024_07/Spectractor 	setup


In [2]:
! eups list -s lsst_distrib

   g4213664e8e+903d45c1e3 	current w_2024_08 setup


In [3]:
import lsst.daf.butler as dafButler
#import lsst.summit.utils.butlerUtils as butlerUtils

In [4]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm


import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.io import fits
from astropy.table import Table
import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

In [5]:
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.time import Time


In [6]:
# LSST Display
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [7]:
from lsst.afw.table import *
from lsst.afw.image import *

In [8]:
dir(lsst.afw.image)

['ApCorrMap',
 'CoaddInputs',
 'Color',
 'DecoratedImage',
 'DecoratedImageD',
 'DecoratedImageF',
 'DecoratedImageI',
 'DecoratedImageL',
 'DecoratedImageU',
 'DefectBase',
 'Exposure',
 'ExposureD',
 'ExposureF',
 'ExposureFitsReader',
 'ExposureI',
 'ExposureInfo',
 'ExposureL',
 'ExposureSummaryStats',
 'ExposureU',
 'FilterLabel',
 'HORIZON',
 'Image',
 'ImageBaseD',
 'ImageBaseF',
 'ImageBaseI',
 'ImageBaseL',
 'ImageBaseU',
 'ImageD',
 'ImageF',
 'ImageFitsReader',
 'ImageI',
 'ImageL',
 'ImageOrigin',
 'ImagePcaD',
 'ImagePcaF',
 'ImagePcaI',
 'ImagePcaL',
 'ImagePcaMD',
 'ImagePcaMF',
 'ImagePcaMI',
 'ImagePcaML',
 'ImagePcaMU',
 'ImagePcaU',
 'ImageSliceD',
 'ImageSliceF',
 'ImageU',
 'IncompleteDataError',
 'LOCAL',
 'MOUNT',
 'Mask',
 'MaskFitsReader',
 'MaskPixel',
 'MaskX',
 'MaskedImage',
 'MaskedImageD',
 'MaskedImageF',
 'MaskedImageFitsReader',
 'MaskedImageI',
 'MaskedImageL',
 'MaskedImageU',
 'Measurement',
 'MultibandExposure',
 'MultibandImage',
 'MultibandMask',

In [9]:
transform = AsinhStretch() + PercentileInterval(99.)

In [10]:
pd.options.display.max_columns = None
#pd.options.display.max_rows = None



## Configuration

In [11]:
DATE = 202402
#filesource_out="finalized_src_table_202402.csv"
#filetractpatch_out = 'Latiss_tractspatchs_202312.pickle'

In [12]:
#repo =  "/sdf/group/rubin/repo/main"
repo = "/sdf/group/rubin/repo/oga/"
butler = dafButler.Butler(repo)
registry = butler.registry

In [13]:
#file_out="sourceTable_202312.csv"

## Give the collection

In [14]:
#my_collection = "LATISS/runs/AUXTEL_DRP_IMAGING_2023-11A-10A-09AB-08ABC-07AB-05AB/w_2023_46/PREOPS-4553"
my_collection = "LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20240201/w_2024_05/PREOPS-4871"

In [15]:
date_work = 202403

## Config

# Sources

In [16]:
# src : a Source catalog
# srcMatchFull
# finalized_src_table : sape of a pandas dataframe

#'isolated_star_cat', {instrument, skymap, tract}, DataFrame),
#'isolated_star_sources', {instrument, skymap, tract}, DataFrame),
#'isolated_star_sources_atlas_refcat2_20220201_match', {skymap, tract}, DataFrame),

data_product = "fgcmVisitCatalog" # OK
data_product = "transmission_atmosphere_fgcm" # write a fits file
data_product = "fgcm_standard_passband"  # does not exist
data_product = "fgcmFitParameters0" # BaseCatalog
data_product = "fgcm_standard_atmosphere" # does not exists
data_product = "fgcmMakeLut_config" # does not ecists
data_product = "fgcmLookUpTable" # OK, but many Nan
data_product = "transmission_atmosphere_fgcm"


# DatasetType('fgcmMakeLut_metadata', {instrument}, TaskMetadata),
# DatasetType('fgcmMakeLut_log', {instrument}, ButlerLogRecords),
# DatasetType('fgcmBuildFromIsolatedStars_config', {}, Config),
# DatasetType('fgcm_star_ids', {instrument}, ArrowAstropy),
# DatasetType('fgcm_reference_stars', {instrument}, ArrowAstropy),
# DatasetType('fgcmVisitCatalog', {instrument}, Catalog),
# DatasetType('fgcm_star_observations', {instrument}, ArrowAstropy),
# DatasetType('fgcmBuildFromIsolatedStars_metadata', {instrument}, TaskMetadata),
# DatasetType('fgcmBuildFromIsolatedStars_log', {instrument}, ButlerLogRecords),
# DatasetType('fgcmFitCycle_config', {}, Config),
# DatasetType('fgcmFlaggedStars0', {instrument}, Catalog),
# DatasetType('fgcmAtmosphereParameters4', {instrument}, Catalog),
# DatasetType('fgcmFitParameters4', {instrument}, Catalog),
# DatasetType('fgcmZeropoints4', {instrument}, Catalog),
# DatasetType('fgcmFlaggedStars1', {instrument}, Catalog),
# DatasetType('fgcmFlaggedStars2', {instrument}, Catalog),
# DatasetType('fgcmStandardStars4', {instrument}, SimpleCatalog),
# DatasetType('fgcmFitParameters3', {instrument}, Catalog),
# DatasetType('fgcmFitParameters1', {instrument}, Catalog),
# DatasetType('fgcmFlaggedStars4', {instrument}, Catalog),
# DatasetType('fgcmFitParameters2', {instrument}, Catalog),
# DatasetType('fgcmFlaggedStars3', {instrument}, Catalog),
# DatasetType('fgcmFitCycle_metadata', {instrument}, TaskMetadata),
# DatasetType('fgcmFitCycle_log', {instrument}, ButlerLogRecords),
# DatasetType('fgcmStandardStars6', {instrument}, SimpleCatalog),
# DatasetType('fgcmZeropoints6', {instrument}, Catalog),
# DatasetType('fgcmFitParameters5', {instrument}, Catalog),
# DatasetType('fgcmAtmosphereParameters6', {instrument}, Catalog),
# DatasetType('fgcmFlaggedStars6', {instrument}, Catalog),
# DatasetType('fgcmFitParameters6', {instrument}, Catalog),
# DatasetType('fgcmFlaggedStars5', {instrument}, Catalog),
# DatasetType('fgcmFlaggedStars8', {instrument}, Catalog),
# DatasetType('fgcmFlaggedStars7', {instrument}, Catalog),
# DatasetType('fgcmFitParameters8', {instrument}, Catalog),
# DatasetType('fgcmFitParameters9', {instrument}, Catalog),
# DatasetType('fgcmFitParameters7', {instrument}, Catalog),
# DatasetType('fgcmFlaggedStars9', {instrument}, Catalog),
# DatasetType('fgcmOutputProducts_metadata', {instrument}, TaskMetadata),
# DatasetType('transmission_atmosphere_fgcm', {band, instrument, physical_filter, visit}, TransmissionCurve),
# DatasetType('fgcmPhotoCalibCatalog', {band, instrument, physical_filter, visit}, ExposureCatalog),
# DatasetType('fgcmOutputProducts_log', {instrument}, ButlerLogRecords),
# DatasetType('fgcmOutputProducts_config', {}, Config)]


datasetRefs = butler.registry.queryDatasets(datasetType=data_product, collections=my_collection, where= "instrument='LATISS'")
filesource_out=f"{data_product}_{DATE}.csv"

In [17]:
all_df = []

for i, ref in enumerate(datasetRefs):
    print(f"========================datasetType = {data_product} ============================================")
    print("fullId.............................:",ref.dataId.full)
    
    data = butler.get(ref)

    if isinstance(data, pd.DataFrame):
        print("pandas dataframe")
        df = data
    elif isinstance(data,Table):
        t = data
        df = t.to_pandas()
    else:
        df = []
        print(type(data),"\n", data.getSchema())
        data.writeFits(f"{data_product}_{date_work}.fits","w")
        if isinstance(data,BaseCatalog):
            t= data.asAstropy()
           

========================datasetType = transmission_atmosphere_fgcm ============================================
fullId.............................: {instrument: 'LATISS', visit: 2023051100262, band: 'r', physical_filter: 'SDSSr_65mm~empty'}


/tmp/ipykernel_68511/2725728736.py:5: FutureWarning: Call to deprecated function (or staticmethod) full. (DataCoordinate.full is deprecated in favor of .mapping, and will be dropped after v27.) -- Deprecated since version v27.
  print("fullId.............................:",ref.dataId.full)


AttributeError: 'lsst.afw.image._imageLib.TransmissionCurve' object has no attribute 'getSchema'

In [ ]:
data

In [ ]:
data.asAstropy()

In [ ]:
t.to_pandas()

In [ ]:
t.to_pandas()

In [ ]:
df = pd.concat(all_df)

In [ ]:
df

In [ ]:
df.to_csv(filesource_out)

In [ ]:
print(df.columns.tolist())

In [ ]:
df['tract'].unique()